In [1]:
import re
import xml.sax.saxutils as saxutils
from bs4 import BeautifulSoup
#https://pypi.org/project/beautifulsoup4/
from gensim.models.word2vec import Word2Vec
#https://radimrehurek.com/gensim/models/word2vec.html
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM

from multiprocessing import cpu_count
#https://pypi.org/project/multiprocess/
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

import numpy as np
import pandas as pd
#https://scikit-learn.org/stable/modules/cross_validation.html
from sklearn.model_selection  import train_test_split

Using TensorFlow backend.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shrey\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shrey\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shrey\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
np.random.seed(1000)

data_folder = 'D:\\text-classification\\'

sgml_number_of_files = 22
sgml_file_name_template = 'reut2-NNN.sgm'
#https://kdd.ics.uci.edu/databases/reuters21578/README.txt
# Category files
category_files = {
    'to_': ('Topics', 'all-topics-strings.lc.txt'),
    'pl_': ('Places', 'all-places-strings.lc.txt'),
    'pe_': ('People', 'all-people-strings.lc.txt'),
    'or_': ('Organizations', 'all-orgs-strings.lc.txt'),
    'ex_': ('Exchanges', 'all-exchanges-strings.lc.txt')
}

#features
num_features = 500
# Limit
document_max_num_words = 100
#categories
selected_categories = ['pl_usa']

In [3]:
# Reading all categories
category_data = []

for category_prefix in category_files.keys():
    with open(data_folder + category_files[category_prefix][1], 'r') as file:
        for category in file.readlines():
            category_data.append([category_prefix + category.strip().lower(), category_files[category_prefix][0],0])

#dataframe
news_categories = pd.DataFrame(data=category_data, columns=['Name', 'Type', 'Newslines'])

In [4]:
def update_frequencies(categories):
    for category in categories:
        idx = news_categories[news_categories.Name == category].index[0]
        f = news_categories.get_value(idx, 'Newslines')
        news_categories.set_value(idx, 'Newslines', f+1)
    
def to_category_vector(categories, target_categories):
    l = len(target_categories)
    vector = np.zeros(l).astype(float)
    
    for i in range(l):
        if target_categories[i] in categories:
            vector[i] = 1.0
    
    return vector

In [5]:
document_X = {}
document_Y = {}

def strip_tags(text):
    return re.sub('<[^<]+?>', '', text).strip()

def unescape(text):
    return saxutils.unescape(text)

# Iterate
for i in range(sgml_number_of_files):
    if i < 10:
        seq = '00' + str(i)
    else:
        seq = '0' + str(i)
        
    file_name = sgml_file_name_template.replace('NNN', seq)
    print('Reading file: %s' % file_name)
    
    with open(data_folder + file_name, 'r') as file:
        content = BeautifulSoup(file.read().lower())
        
        for newsline in content('reuters'):
            document_categories = []
            
            #Id
            document_id = newsline['newid']
            
            #text
            document_body = strip_tags(str(newsline('text')[0].body)).replace('reuter\n&#3;', '')
            document_body = unescape(document_body)
            
            #categories
            topics = newsline.topics.contents
            places = newsline.places.contents
            people = newsline.people.contents
            orgs = newsline.orgs.contents
            exchanges = newsline.exchanges.contents
            
            for topic in topics:
                document_categories.append('to_' + strip_tags(str(topic)))
                
            for place in places:
                document_categories.append('pl_' + strip_tags(str(place)))
                
            for person in people:
                document_categories.append('pe_' + strip_tags(str(person)))
                
            for org in orgs:
                document_categories.append('or_' + strip_tags(str(org)))
                
            for exchange in exchanges:
                document_categories.append('ex_' + strip_tags(str(exchange)))
                
            # Create new document    
            update_frequencies(document_categories)
            
            document_X[document_id] = document_body
            document_Y[document_id] = to_category_vector(document_categories, selected_categories)

Reading file: reut2-000.sgm


c:\users\shrey\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.
c:\users\shrey\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """


Reading file: reut2-001.sgm
Reading file: reut2-002.sgm
Reading file: reut2-003.sgm
Reading file: reut2-004.sgm
Reading file: reut2-005.sgm
Reading file: reut2-006.sgm
Reading file: reut2-007.sgm
Reading file: reut2-008.sgm
Reading file: reut2-009.sgm
Reading file: reut2-010.sgm
Reading file: reut2-011.sgm
Reading file: reut2-012.sgm
Reading file: reut2-013.sgm
Reading file: reut2-014.sgm
Reading file: reut2-015.sgm
Reading file: reut2-016.sgm
Reading file: reut2-017.sgm
Reading file: reut2-018.sgm
Reading file: reut2-019.sgm
Reading file: reut2-020.sgm
Reading file: reut2-021.sgm


In [6]:
news_categories.sort_values(by='Newslines', ascending=False, inplace=True)
print(news_categories.head(20))

                Name           Type  Newslines
296           pl_usa         Places      12542
35           to_earn         Topics       3987
0             to_acq         Topics       2448
293            pl_uk         Places       1489
219         pl_japan         Places       1138
166        pl_canada         Places       1104
73       to_money-fx         Topics        801
28          to_crude         Topics        634
45          to_grain         Topics        628
302  pl_west-germany         Places        567
126         to_trade         Topics        552
55       to_interest         Topics        513
191        pl_france         Places        469
587            or_ec  Organizations        349
158        pl_brazil         Places        332
130         to_wheat         Topics        306
108          to_ship         Topics        305
145     pl_australia         Places        270
19           to_corn         Topics        254
172         pl_china         Places        223


In [7]:

# Load stop-words
stop_words = set(stopwords.words('english'))

# Initialize tokenizer
tokenizer = RegexpTokenizer('[\'a-zA-Z]+')

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

newsline_documents = []

In [8]:
def tokenize(document):
    words = []

    for sentence in sent_tokenize(document):
        tokens = [lemmatizer.lemmatize(t.lower()) for t in tokenizer.tokenize(sentence) if t.lower() not in stop_words]
        words += tokens

    return words

for key in document_X.keys():
    newsline_documents.append(tokenize(document_X[key]))

number_of_documents = len(document_X)

In [9]:
w2v_model = Word2Vec.load(data_folder + 'reuters.word2vec')

c:\users\shrey\appdata\local\programs\python\python37\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [10]:
# Create new Gensim Word2Vec model
#https://radimrehurek.com/gensim/models/word2vec.html
#https://code.google.com/archive/p/word2vec/
w2v_model = Word2Vec(newsline_documents, size=num_features, min_count=1, window=10, workers=cpu_count())
w2v_model.init_sims(replace=True)
w2v_model.save(data_folder + 'reuters.word2vec')

c:\users\shrey\appdata\local\programs\python\python37\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "
c:\users\shrey\appdata\local\programs\python\python37\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [11]:
num_categories = len(selected_categories)
X = np.zeros(shape=(number_of_documents, document_max_num_words, num_features)).astype(float)
Y = np.zeros(shape=(number_of_documents, num_categories)).astype(float)

empty_word = np.zeros(num_features).astype(float)

for idx, document in enumerate(newsline_documents):
    for jdx, word in enumerate(document):
        if jdx == document_max_num_words:
            break
            
        else:
            if word in w2v_model:
                X[idx, jdx, :] = w2v_model[word]
            else:
                X[idx, jdx, :] = empty_word

for idx, key in enumerate(document_Y.keys()):
    Y[idx, :] = document_Y[key]

c:\users\shrey\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
c:\users\shrey\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

In [14]:
model = Sequential()

model.add(LSTM(int(document_max_num_words*1.5), input_shape=(document_max_num_words, num_features)))
model.add(Dropout(0.3))
model.add(Dense(num_categories))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
# Train model
model.fit(X_train, Y_train, batch_size=512, epochs=5, validation_data=(X_test, Y_test))

# Evaluate model
score, acc = model.evaluate(X_test, Y_test, batch_size=512)
    
print('Score: %1.4f' % score)
print('Accuracy: %1.4f' % acc)

Instructions for updating:
Use tf.cast instead.
Train on 15104 samples, validate on 6474 samples
Epoch 1/5
15104/15104 [==============================] - 1593s 105ms/step - loss: 0.5959 - acc: 0.6751 - val_loss: 0.4102 - val_acc: 0.8706
Epoch 2/5
15104/15104 [==============================] - 1961s 130ms/step - loss: 0.4249 - acc: 0.8599 - val_loss: 0.4357 - val_acc: 0.8462
Epoch 3/5
15104/15104 [==============================] - 2448s 162ms/step - loss: 0.4377 - acc: 0.8424 - val_loss: 0.4586 - val_acc: 0.8268
Epoch 4/5
15104/15104 [==============================] - 1262s 84ms/step - loss: 0.4446 - acc: 0.8398 - val_loss: 0.4435 - val_acc: 0.8483
Epoch 5/5
6474/6474 [==============================] - 12s 2ms/step
Score: 0.4447
Accuracy: 0.8133
